In [2]:
from pymongo import MongoClient
import pandas as pd
import numpy as np
client = MongoClient()
db = client.openfoodfacts
collection = db.products

In [6]:
def process_data(data) :
    
    max_salt_content = []
    for l in data['salt_content'] :
        if l != None and len(l) > 0 and 'percent_max' in l[0]  :
            max_salt_content.append(l[0]['percent_max'])
        else :
            max_salt_content.append(0)

    data['max_salt_content'] = max_salt_content

    min_salt_content = []
    for l in data['salt_content'] :
        if l != None and len(l) > 0 and 'percent_min' in l[0]  :
            min_salt_content.append(l[0]['percent_min'])
        else :
            min_salt_content.append(0)

    data['min_salt_content'] = min_salt_content

    max_oil_content = []
    for l in data['oil_content'] :
        if l != None and len(l) > 0 and 'percent_max' in l[0]  :
            max_oil_content.append(l[0]['percent_max'])
        else :
            max_oil_content.append(0)

    data['max_oil_content'] = max_oil_content

    min_oil_content = []
    for l in data['oil_content'] :
        if l != None and len(l) > 0 and 'percent_min' in l[0]  :
            min_oil_content.append(l[0]['percent_min'])
        else :
            min_oil_content.append(0)

    data['min_oil_content'] = min_oil_content

    oil_type = []
    for l in data['oil_content'] :
        if l != None and len(l) > 0 and 'percent_min' in l[0]  :
            oil_type.append(l[0]['text'])
        else :
            oil_type.append("")

    data['oil_type'] = oil_type


    ##################################################################
    ######    extraction of _100g
    ##################################################################

    energy_100g = []
    for l in data['nutriments'] :
        if len(l) > 0 and 'energy_100g' in l :
            energy_100g.append(l['energy_100g'])
        else :
            energy_100g.append(None)

    data['energy_100g'] = energy_100g


    energy_kcal_100g = []
    for l in data['nutriments'] :
        if len(l) > 0 and 'energy-kcal_100g' in l :
            energy_kcal_100g.append(l['energy-kcal_100g'])
        else :
            energy_kcal_100g.append(None)

    data['energy_kcal_100g'] = energy_kcal_100g


    saturated_fat_100g = []
    for l in data['nutriments'] :
        if len(l) > 0 and 'saturated-fat_100g' in l :
            saturated_fat_100g.append(l['saturated-fat_100g'])
        else :
            saturated_fat_100g.append(None)

    data['saturated_fat_100g'] = saturated_fat_100g


    fat_100g = []
    for l in data['nutriments'] :
        if len(l) > 0 and 'fat_100g' in l :
            fat_100g.append(l['fat_100g'])
        else :
            fat_100g.append(None)

    data['fat_100g'] = fat_100g


    carbohydrates_100g = []
    for l in data['nutriments'] :
        if len(l) > 0 and 'carbohydrates_100g' in l :
            carbohydrates_100g.append(l['carbohydrates_100g'])
        else :
            carbohydrates_100g.append(None)

    data['carbohydrates_100g'] = carbohydrates_100g


    sugars_100g = []
    for l in data['nutriments'] :
        if len(l) > 0 and 'sugars_100g' in l :
            sugars_100g.append(l['sugars_100g'])
        else :
            sugars_100g.append(None)

    data['sugars_100g'] = sugars_100g


    fruits_vegetables_nuts_estimate_from_ingredients_100g = []
    for l in data['nutriments'] :
        if len(l) > 0 and 'fruits-vegetables-nuts-estimate-from-ingredients_100g' in l :
            fruits_vegetables_nuts_estimate_from_ingredients_100g.append(l['fruits-vegetables-nuts-estimate-from-ingredients_100g'])
        else :
            fruits_vegetables_nuts_estimate_from_ingredients_100g.append(None)

    data['fruits_vegetables_nuts_estimate_from_ingredients_100g'] = fruits_vegetables_nuts_estimate_from_ingredients_100g


    proteins_100g = []
    for l in data['nutriments'] :
        if len(l) > 0 and 'proteins_100g' in l :
            proteins_100g.append(l['proteins_100g'])
        else :
            proteins_100g.append(None)

    data['proteins_100g'] = proteins_100g


    sodium_100g = []
    for l in data['nutriments'] :
        if len(l) > 0 and 'sodium_100g' in l :
            sodium_100g.append(l['sodium_100g'])
        else :
            sodium_100g.append(None)

    data['sodium_100g'] = sodium_100g



    salt_100g = []
    for l in data['nutriments'] :
        if len(l) > 0 and 'salt_100g' in l :
            salt_100g.append(l['salt_100g'])
        else :
            salt_100g.append(None)

    data['salt_100g'] = salt_100g


    nutrition_score_fr_100g = []
    for l in data['nutriments'] :
        if len(l) > 0 and 'nutrition-score-fr_100g' in l :
            nutrition_score_fr_100g.append(l['nutrition-score-fr_100g'])
        else :
            nutrition_score_fr_100g.append(None)

    data['nutrition_score_fr_100g'] = nutrition_score_fr_100g


    fiber_100g = []
    for l in data['nutriments'] :
        if len(l) > 0 and 'fiber_100g' in l :
            fiber_100g.append(l['fiber_100g'])
        else :
            fiber_100g.append(None)

    data['fiber_100g'] = fiber_100g


    nova_group_100g = []
    for l in data['nutriments'] :
        if len(l) > 0 and 'nova-group_100g' in l :
            nova_group_100g.append(l['nova-group_100g'])
        else :
            nova_group_100g.append(None)

    data['nova_group_100g'] = nova_group_100g



In [146]:
proj = { 
    'product_name_fr' : 1,
    'product_name' : 1,
    'interface_version_modified' : 1,
    'ingredients_text' : 1, 
    'ingredients_n' : 1,
    'unknown_ingredients' : 1,
    'serving_size' : 1,
    'serving_quantity' : 1,
    'countries_hierarchy' : 1,
    'manufacturing_places' : 1,
    'cities_tags' : 1,
    'category_properties' : 1,
    'categories' : 1,
    'pnns_groups_1' : 1,
    'pnns_groups_2' : 1,
    'nutriments' : 1,
    'compared_to_category' : 1,
    'ingredients' : 1, 
    'salt_content' : 1, 
    'oil_content' : 1, 
}

addFields = { 
    
    'ingredients' : "$ingredients.text",
    
    'salt_content' : {
            '$filter' : {
               'input' : "$ingredients",
               'as' : "item",
               'cond' : { '$or' : [ 
                   { '$eq' : [ "$$item.text", 'Sel' ] }, 
                   { '$eq' : [ "$$item.text", 'sel' ] },
                   { '$eq' : [ "$$item.text", 'SEL' ] },
                   { '$eq' : [ "$$item.text", 'VINAGE SEL' ] },
                   { '$eq' : [ "$$item.text", 'sel marin' ] },
                   { '$eq' : [ "$$item.text", 'Sel marin' ] },
                   { '$eq' : [ "$$item.text", 'salt' ] },
                   { '$eq' : [ "$$item.text", 'Salt' ] }] }
            }
    },
    
    
    'oil_content': {
            '$filter' : {
               'input' : "$ingredients",
               'as' : "item",
               'cond' : { '$or' : [ { 
                   '$eq' : [ "$$item.text", 'huile d\'olive extra vierge' ] }, {
                   '$eq' : [ "$$item.text", 'huile extra vierge d\'olive' ] }, { 
                   '$eq' : [ "$$item.text", 'huile d\'olive extra-vierge' ] }, { 
                   '$eq' : [ "$$item.text", 'Huile d\'olive vierge extra' ] }, { 
                   '$eq' : [ "$$item.text", 'Huile d\'olive extra vierge' ] }, {
                   '$eq' : [ "$$item.text", 'huile dolive* vierge exta' ] }, { 
                   '$eq' : [ "$$item.text", 'huile d\'olive vierge extraite à froid' ] }, { 
                   '$eq' : [ "$$item.text", 'huile d\'olive vierge' ] }, {
                   '$eq' : [ "$$item.text", 'huile d\'olive' ] }, {
                   '$eq' : [ "$$item.text", 'Huile d\'olive' ] }, {
                   '$eq' : [ "$$item.text", 'huile de colza' ] }, { 
                   '$eq' : [ "$$item.text", 'huile d\'olive vierge extra extraite à froid' ] }, { 
                   '$eq' : [ "$$item.text", 'huile de tournesol' ] }, {
                   '$eq' : [ "$$item.text", 'Huile de tournesol' ] }, {
                   '$eq' : [ "$$item.text", 'huile de tournesol désodorisée' ] }, {
                   '$eq' : [ "$$item.text", 'Extra Virgin Olive Oil' ] }, { 
                   '$eq' : [ "$$item.text", 'Huile de palme' ] }, {
                   '$eq' : [ "$$item.text", 'natives Olivenöl extra' ] }, { 
                   '$eq' : [ "$$item.text", 'Sonnenblumenöl' ] }, { 
                   '$eq' : [ "$$item.text", 'Olivenöl' ] }, { 
                   '$eq' : [ "$$item.text", 'Huile Végétale' ] }, {
                   '$eq' : [ "$$item.text", 'Huile de colza' ] }, {
                   '$eq' : [ "$$item.text", 'huile végétale de colza' ] }, { 
                   '$eq' : [ "$$item.text", 'Huile végétale de colza' ] }, {
                   '$eq' : [ "$$item.text", 'Huile d\'olive vierge extra' ] } ] 
                    }
            }
    },
}




match = {
    
    'ingredients' : { 
        "$elemMatch" : { 
            "id" : { '$regex' : "tomate*", '$options' : "i" },
        } 
    },
    
    "product_name_fr" : { '$regex' : "sauce*", '$options' : "i" }
    
}



c_f = collection.aggregate([
    { '$match' : match } ,
    { '$project' : proj },
    { '$addFields' : addFields }
])

data = pd.DataFrame(list(c_f))

In [147]:
data.head(5)

,_id,nutriments,product_name,categories,pnns_groups_2,interface_version_modified,compared_to_category,pnns_groups_1,ingredients_n,ingredients,category_properties,product_name_fr,ingredients_text,countries_hierarchy,salt_content,oil_content,cities_tags,serving_quantity,manufacturing_places,serving_size
0,0000069162176,"{'salt_unit': '', 'energy': 586, 'proteins': 4...",Quenelles de brochet sauce Nantua,"Plats préparés, Quenelles, Quenelles de poisso...",One-dish meals,20150316.jqm2,en:pike-quenelles,Composite foods,30,"[INGREDIENTES, agua, harina de tngo tierno car...",{},Quenelles de brochet sauce Nantua,"INGREDIENTES: Quenelles de lucio 43% (huevo, a...",[en:france],[],[],NaN,NaN,NaN,NaN
1,00005010,"{'energy-kcal_value': 106, 'sodium_100g': 0.16...",Organic tomato sauce bolognese,"Epicerie, Produits à la viande, Sauces, Sauces...",Dressings and sauces,20150316.jqm2,en:bolognese-sauces,Fat and sauces,19,"[Tomato puree, beef mince meat, carrots, extra...",{'ciqual_food_name:fr': 'Sauce tomate à la via...,Organic tomato sauce bolognese,"Tomato puree* (65,79%), beef mince meat* (19,3...",[en:belgium],"[{'id': 'en:salt', 'text': 'salt', 'vegetarian...",[],NaN,NaN,NaN,NaN
2,00021036,"{'salt': 0.6, 'fiber_100g': 3.1, 'carbohydrate...",Tomato & Gorgonzola pasta sauce,,unknown,20150316.jqm2,NaN,unknown,13,"[sauce à la tomate, à Itoignon kJ et à la caro...",{},Tomato & Gorgonzola pasta sauce,"sauce à la tomate, à Itoignon kJ et à la carot...",[en:france],[],[],NaN,NaN,NaN,NaN
3,00089227,"{'fiber': 1.7, 'fruits-vegetables-nuts-estimat...",Tomato & Sausage Pasta Sauce,"Epicerie, Produits à la viande, Sauces, Sauces...",Dressings and sauces,20120622,en:tomato-sauces,Fat and sauces,19,"[Pulpe de tomate, Saucisse de porc, Concentré ...",{},Tomato & Sausage Pasta Sauce,Pulpe de tomate (57 %) Saucisse de porc (24 %)...,[en:france],"[{'id': 'en:salt', 'vegan': 'yes', 'text': 'Se...","[{'from_palm_oil': 'no', 'id': 'en:extra-virgi...",[],170,Italie,170 g e
4,0011110791474,"{'sodium_100g': 2, 'energy-kcal_value': 100, '...",Peri peri chili culinary hot sauce,"Groceries, Sauces",Dressings and sauces,20150316.jqm2,en:sauces,Fat and sauces,15,"[Water, red bell peppers, distilled vinegar, t...",{},Peri peri chili culinary hot sauce,"Water, red bell peppers, distilled vinegar, to...","[en:france, en:united-states]","[{'percent_min': 0, 'rank': 6, 'percent_max': ...",[],NaN,5,NaN,1 tsp (5 ml)


In [148]:
process_data(data)

In [149]:
data.head()

,_id,nutriments,product_name,categories,pnns_groups_2,interface_version_modified,compared_to_category,pnns_groups_1,ingredients_n,ingredients,...,fat_100g,carbohydrates_100g,sugars_100g,fruits_vegetables_nuts_estimate_from_ingredients_100g,proteins_100g,sodium_100g,salt_100g,nutrition_score_fr_100g,fiber_100g,nova_group_100g
0,0000069162176,"{'salt_unit': '', 'energy': 586, 'proteins': 4...",Quenelles de brochet sauce Nantua,"Plats préparés, Quenelles, Quenelles de poisso...",One-dish meals,20150316.jqm2,en:pike-quenelles,Composite foods,30,"[INGREDIENTES, agua, harina de tngo tierno car...",...,11.0,5.1,0.6,0.0,4.6,1.960,4.90,16.0,0.5,4.0
1,00005010,"{'energy-kcal_value': 106, 'sodium_100g': 0.16...",Organic tomato sauce bolognese,"Epicerie, Produits à la viande, Sauces, Sauces...",Dressings and sauces,20150316.jqm2,en:bolognese-sauces,Fat and sauces,19,"[Tomato puree, beef mince meat, carrots, extra...",...,8.0,3.4,3.2,78.9,4.2,0.160,0.40,-1.0,NaN,NaN
2,00021036,"{'salt': 0.6, 'fiber_100g': 3.1, 'carbohydrate...",Tomato & Gorgonzola pasta sauce,,unknown,20150316.jqm2,NaN,unknown,13,"[sauce à la tomate, à Itoignon kJ et à la caro...",...,6.8,3.8,3.5,3.5,3.8,0.240,0.60,NaN,3.1,NaN
3,00089227,"{'fiber': 1.7, 'fruits-vegetables-nuts-estimat...",Tomato & Sausage Pasta Sauce,"Epicerie, Produits à la viande, Sauces, Sauces...",Dressings and sauces,20120622,en:tomato-sauces,Fat and sauces,19,"[Pulpe de tomate, Saucisse de porc, Concentré ...",...,8.4,9.1,4.6,63.0,4.3,0.316,0.79,3.0,1.7,3.0
4,0011110791474,"{'sodium_100g': 2, 'energy-kcal_value': 100, '...",Peri peri chili culinary hot sauce,"Groceries, Sauces",Dressings and sauces,20150316.jqm2,en:sauces,Fat and sauces,15,"[Water, red bell peppers, distilled vinegar, t...",...,0.0,0.0,0.0,0.0,0.0,2.000,5.00,NaN,NaN,4.0


In [150]:
data.to_csv('O_produits.csv')

In [10]:
EAN13_temp = pd.read_csv('N_NIELSEN/N_EAN13.csv')['3760052155536'].astype('str').values

EAN13 = []
for ean in EAN13_temp :
    EAN13.append(ean)


match = {
    
    "_id" : { "$in" : EAN13 }
    
}

proj = { 
    'product_name_fr' : 1,
    'product_name' : 1,
    'interface_version_modified' : 1,
    'ingredients_text' : 1, 
    'ingredients_n' : 1,
    'unknown_ingredients' : 1,
    'serving_size' : 1,
    'serving_quantity' : 1,
    'countries_hierarchy' : 1,
    'manufacturing_places' : 1,
    'cities_tags' : 1,
    'category_properties' : 1,
    'categories' : 1,
    'pnns_groups_1' : 1,
    'pnns_groups_2' : 1,
    'nutriments' : 1,
    'compared_to_category' : 1,
    'ingredients' : 1, 
    'salt_content' : 1, 
    'oil_content' : 1, 
}

addFields = { 
    
    'ingredients' : "$ingredients.text",
    
    'salt_content' : {
            '$filter' : {
               'input' : "$ingredients",
               'as' : "item",
               'cond' : { '$or' : [ 
                   { '$eq' : [ "$$item.text", 'Sel' ] }, 
                   { '$eq' : [ "$$item.text", 'sel' ] },
                   { '$eq' : [ "$$item.text", 'SEL' ] },
                   { '$eq' : [ "$$item.text", 'VINAGE SEL' ] },
                   { '$eq' : [ "$$item.text", 'sel marin' ] },
                   { '$eq' : [ "$$item.text", 'Sel marin' ] },
                   { '$eq' : [ "$$item.text", 'salt' ] },
                   { '$eq' : [ "$$item.text", 'Salt' ] }] }
            }
    },
    
    
    'oil_content': {
            '$filter' : {
               'input' : "$ingredients",
               'as' : "item",
               'cond' : { '$or' : [ { 
                   '$eq' : [ "$$item.text", 'huile d\'olive extra vierge' ] }, {
                   '$eq' : [ "$$item.text", 'huile extra vierge d\'olive' ] }, { 
                   '$eq' : [ "$$item.text", 'huile d\'olive extra-vierge' ] }, { 
                   '$eq' : [ "$$item.text", 'Huile d\'olive vierge extra' ] }, { 
                   '$eq' : [ "$$item.text", 'Huile d\'olive extra vierge' ] }, {
                   '$eq' : [ "$$item.text", 'huile dolive* vierge exta' ] }, { 
                   '$eq' : [ "$$item.text", 'huile d\'olive vierge extraite à froid' ] }, { 
                   '$eq' : [ "$$item.text", 'huile d\'olive vierge' ] }, {
                   '$eq' : [ "$$item.text", 'huile d\'olive' ] }, {
                   '$eq' : [ "$$item.text", 'Huile d\'olive' ] }, {
                   '$eq' : [ "$$item.text", 'huile de colza' ] }, { 
                   '$eq' : [ "$$item.text", 'huile d\'olive vierge extra extraite à froid' ] }, { 
                   '$eq' : [ "$$item.text", 'huile de tournesol' ] }, {
                   '$eq' : [ "$$item.text", 'Huile de tournesol' ] }, {
                   '$eq' : [ "$$item.text", 'huile de tournesol désodorisée' ] }, {
                   '$eq' : [ "$$item.text", 'Extra Virgin Olive Oil' ] }, { 
                   '$eq' : [ "$$item.text", 'Huile de palme' ] }, {
                   '$eq' : [ "$$item.text", 'natives Olivenöl extra' ] }, { 
                   '$eq' : [ "$$item.text", 'Sonnenblumenöl' ] }, { 
                   '$eq' : [ "$$item.text", 'Olivenöl' ] }, { 
                   '$eq' : [ "$$item.text", 'Huile Végétale' ] }, {
                   '$eq' : [ "$$item.text", 'Huile de colza' ] }, {
                   '$eq' : [ "$$item.text", 'huile végétale de colza' ] }, { 
                   '$eq' : [ "$$item.text", 'Huile végétale de colza' ] }, {
                   '$eq' : [ "$$item.text", 'Huile d\'olive vierge extra' ] } ] 
                    }
            }
    },
}

c_f = collection.aggregate([
    { '$match' : match } ,
    { '$project' : proj },
    { '$addFields' : addFields },
])

data_EAN13 = pd.DataFrame(list(c_f))

In [11]:
data_EAN13.head()

,_id,categories,ingredients,nutriments,product_name,manufacturing_places,category_properties,ingredients_text,product_name_fr,interface_version_modified,ingredients_n,pnns_groups_1,pnns_groups_2,cities_tags,compared_to_category,countries_hierarchy,salt_content,oil_content,serving_quantity,serving_size
0,3017239003969,"Epicerie, Sauces, Sauces aux légumes, Sauces a...","[Tomate, olive noire, huile d'olive, câpre, vi...","{'energy-kcal_100g': 122, 'carbohydrates_value...",Sauce aux Olives Provençale,France,{},"Tomate (51%), olive noire (36%), huile d'olive...",Sauce aux Olives Provençale,20150316.jqm2,11,Fat and sauces,Dressings and sauces,[],en:provencale-sauces,[en:france],"[{'percent_min': 0, 'rank': 8, 'vegan': 'yes',...","[{'id': 'en:olive-oil', 'text': 'huile d'olive...",NaN,NaN
1,3017239003976,"Epicerie, Sauces, Sauces aux légumes, Sauces a...","[olive verte égouttée, huile de tournesol, bas...","{'nova-group_100g': 4, 'energy-kj_100g': 1086,...",Sauce aux olives façon pesto vert,,{},"olive verte égouttée (33%), huile de tournesol...",Sauce aux olives façon pesto vert,20120622,27,Fat and sauces,Dressings and sauces,[],en:vegetable-sauces,[en:france],"[{'percent_min': 0, 'rank': 15, 'vegan': 'yes'...","[{'text': 'huile de tournesol', 'id': 'en:sunf...",NaN,NaN
2,3017239003983,"Epicerie, Sauces, Sauces Pesto, Sauces Pesto R...","[Tomate, olive, huile de tournesol, pomme de t...","{'nutrition-score-fr_serving': 7, 'saturated-f...",Sauce aux olives façon pesto rosso,France,"{'ciqual_food_name:fr': 'Sauce pesto rosso, pr...","Tomate 46%, olive 18% (dont olive verte égoutt...",Sauce aux olives façon pesto rosso,20120622,31,Fat and sauces,Dressings and sauces,[],en:red-pestos,[en:france],"[{'rank': 15, 'vegan': 'yes', 'vegetarian': 'y...","[{'vegetarian': 'yes', 'rank': 3, 'vegan': 'ye...",190,190g
3,3017239003990,NaN,[],"{'saturated-fat': 1, 'sodium_unit': 'g', 'satu...",Sauce aux Olives Citron Confit,NaN,{},NaN,NaN,20150316.jqm2,NaN,unknown,unknown,NaN,NaN,[en:france],[],[],NaN,NaN
4,3017239004003,"Epicerie, Sauces, Sauces tomate, Sauces tomate...","[Tomate, champignon, olive verte égouttée, pom...","{'carbohydrates': 4.1, 'proteins_100g': 1.5, '...",Sauce aux olives champignons,France,"{'ciqual_food_name:en': 'Tomato sauce, w mushr...","Tomate (36 %), champignon (24 %), olive verte ...",Sauce aux olives champignons,20150316.jqm2,16,Fat and sauces,Dressings and sauces,[],fr:sauces-tomates-aux-champignons,[en:france],"[{'percent_min': 0, 'vegetarian': 'yes', 'rank...","[{'percent_min': 0, 'from_palm_oil': 'no', 'ra...",NaN,NaN


In [12]:
process_data(data_EAN13)

In [15]:
data_EAN13.head()['oil_content'][0]

[{'id': 'en:olive-oil',
  'text': "huile d'olive",
  'vegetarian': 'yes',
  'percent_max': 13,
  'vegan': 'yes',
  'rank': 3,
  'percent_min': 2.1666666666666665,
  'from_palm_oil': 'no'}]

In [155]:
data_EAN13.to_csv('O_produits_EAN13.csv')

In [3]:
EAN13_temp = pd.read_csv('N_NIELSEN/N_EAN13.csv')

In [4]:
EAN13_temp

,3760052155536
0,8854404005213
1,8858135071028
2,3760052155512
3,3416952625159
4,3416360008261
...,...
1419,3038359002823
1420,3038359008498
1421,3038359005657
1422,3038359001413
